In [54]:
#Обрабатываем данные для загрузки в БД

In [55]:
#Импортируем нужные библиотеки
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
import random
import string

In [56]:
#Получаем доступ к данным из локальных файлов
path_to_files = 'C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/'
card_base = pd.read_csv(path_to_files + 'Cardbase.csv', delimiter = ',')
customers = pd.read_csv(path_to_files + 'CustomerBase.csv', delimiter = ',')
fraud = pd.read_csv(path_to_files + 'FraudBase.csv', delimiter = ',')
transactions = pd.read_csv(path_to_files + 'TransactionBase.csv', delimiter=',')

In [57]:
#Создаём таблицу с тарифами
tariffs = {'Card_Family':['Premium','Gold', 'Platinum'], 
           'Interest rate':[0.2,0.17,0.15],
           'Maintenance (year)':[0,99,199],
           'Withdraw %':[0.059,0.059,0.059],
           'Withdraw rub':[99,99,99],
           'no % period':[50,120,180]}
card_tarrif = pd.DataFrame(tariffs)
card_tarrif

,Card_Family,Interest rate,Maintenance (year),Withdraw %,Withdraw rub,no % period
0,Premium,0.20,0,0.059,99,50
1,Gold,0.17,99,0.059,99,120
2,Platinum,0.15,199,0.059,99,180


In [58]:
#Заменяем данные по сегментам на условные типы операций
transactions.loc[(transactions.Transaction_Segment == 'SEG13') | (transactions.Transaction_Segment == 'SEG14'),'Transaction_Segment'] = 'Withdraw'
transactions.loc[(transactions.Transaction_Segment == 'SEG15') 
                 | (transactions.Transaction_Segment == 'SEG16') 
                 | (transactions.Transaction_Segment == 'SEG17') 
                 | (transactions.Transaction_Segment == 'SEG18')
                 | (transactions.Transaction_Segment == 'SEG19')
                 | (transactions.Transaction_Segment == 'SEG20')
                 | (transactions.Transaction_Segment == 'SEG21')
                 | (transactions.Transaction_Segment == 'SEG22')
                 | (transactions.Transaction_Segment == 'SEG23'),'Transaction_Segment'] = 'Refill'
transactions.loc[(transactions.Transaction_Segment != 'Refill') & (transactions.Transaction_Segment != 'Withdraw'), 'Transaction_Segment'] = 'Payment'
transactions = transactions.rename(columns={'Transaction_Segment':'Operation_type'})
transactions['Category'] = ''
for i in range(len(transactions)):
    transactions.iloc[i,5] = np.where(transactions.iloc[i,4]=='Payment', random.randint(1,4), '')
transactions.loc[(transactions.Category == '1','Category')] = 'medicine'
transactions.loc[(transactions.Category == '2','Category')] = 'supermarkets'
transactions.loc[(transactions.Category == '3','Category')] = 'transport'
transactions.loc[(transactions.Category == '4','Category')] = 'house, renovation'


In [59]:
#Создаём функцию замены названия месяца на порядковый номер
def month_replacer(x):
    if x=='Oct':
        value = '10'
    elif x=='Mar':
        value='03' 
    elif x=='May':
        value = '05'
    elif x=='Nov':
        value = '11'
    elif x== 'Jul':
        value = '07'
    elif x== 'Aug':
        value='08'
    elif x=='Jun':
        value='06'
    elif x=='Jan':
        value='01' 
    elif x=='Apr':
        value='04'
    elif x=='Dec':
        value='12'
    elif x=='Sep':
        value='09'
    elif x=='Feb':
        value='02'
    return value
#Преобразуем столбцы с транзакциями в нужный для анализа вид
transactions[['day','month','year']] = transactions['Transaction_Date'].str.split('-',expand=True)
transactions['month'] = transactions['month'].apply(month_replacer)
transactions['year'] = transactions['year'].astype(str)
transactions['day'] = transactions['day'].apply(lambda x: '0'+x if len(x)<2 else x)
transactions['day'] = transactions['day'].astype(str)
transactions['Oper_date'] = '20'+transactions['year']+'-'+transactions['month']+'-'+transactions['day']
transactions = transactions[['Transaction_ID','Oper_date','Credit_Card_ID','Transaction_Value','Operation_type','Category']]
transactions.Oper_date = transactions.Oper_date.apply(lambda _: datetime.strptime(_,"%Y-%m-%d"))
transactions

,Transaction_ID,Oper_date,Credit_Card_ID,Transaction_Value,Operation_type,Category
0,CTID28830551,2016-04-24,1629-9566-3285-2123,23649,Payment,medicine
1,CTID45504917,2016-02-11,3697-6001-4909-5350,26726,Refill,
2,CTID47312290,2016-11-01,5864-4475-3659-1440,22012,Withdraw,
3,CTID25637718,2016-01-28,5991-4421-8476-3804,37637,Refill,
4,CTID66743960,2016-03-17,1893-8853-9900-8478,5113,Withdraw,
...,...,...,...,...,...,...
9995,CTID25037573,2016-06-10,9157-2802-8374-1145,44280,Refill,
9996,CTID43832298,2016-05-23,7416-4529-6690-5703,27163,Payment,"house, renovation"
9997,CTID56532072,2016-08-06,6699-2639-4522-6219,36424,Withdraw,
9998,CTID88101446,2016-10-10,8341-5263-4582-7396,8464,Withdraw,


In [60]:
#Проверяем данные на отсутствующие значения
customers.isna().sum()
card_base.isna().sum()
fraud.isna().sum()
transactions.isna().sum()

Transaction_ID       0
Oper_date            0
Credit_Card_ID       0
Transaction_Value    0
Operation_type       0
Category             0
dtype: int64

In [61]:
#Добавим к таблице с картами данные по дате открытия и закрытия карты
card_base['month started'] = ''
card_base['year started'] = ''
card_base['date_start'] = ''
card_base['expiry_date'] = ''
for i in range(len(card_base)):
    card_base.iloc[i,4]=random.randint(6,12)
    card_base.iloc[i,5]=15
    card_base.iloc[i,6]=str(card_base.iloc[i,4])+'/'+str(card_base.iloc[i,5])
    card_base.iloc[i,7]=str(card_base.iloc[i,4])+'/'+str(card_base.iloc[i,5]+5)
card_base = card_base.loc[:,['Card_Number','Card_Family','Credit_Limit','Cust_ID','date_start','expiry_date']]
card_base

,Card_Number,Card_Family,Credit_Limit,Cust_ID,date_start,expiry_date
0,8638-5407-3631-8196,Premium,530000,CC67088,12/15,12/20
1,7106-4239-7093-1515,Gold,18000,CC12076,8/15,8/20
2,6492-5655-8241-3530,Premium,596000,CC97173,7/15,7/20
3,2868-5606-5152-5706,Gold,27000,CC55858,9/15,9/20
4,1438-6906-2509-8219,Platinum,142000,CC90518,9/15,9/20
...,...,...,...,...,...,...
495,4708-4407-9601-6022,Premium,691000,CC64993,7/15,7/20
496,6042-2856-7280-2925,Gold,33000,CC26787,10/15,10/20
497,8706-3809-5167-3899,Premium,144000,CC32532,12/15,12/20
498,9540-8558-5897-5046,Premium,830000,CC90246,9/15,9/20


In [62]:
#Добавляем информацию о клиентах
customers['Gender'] = ''
customers['Marital_status'] = ''
customers['City'] = ''

def city(l):
    if l==1:
        city = 'Moscow'
    if l==2:
        city = 'Saint Petersburg'
    return city

for i in range(len(customers)):
    customers.iloc[i,4]=random.randint(0,1)
    customers.iloc[i,5]=random.randint(1,2)
    customers.iloc[i,6]= city(random.randint(1,2))
customers = customers.loc[:,['Cust_ID','Age','Gender','Marital_status','City','Customer_Segment']]
customers.Gender = customers.Gender.apply(lambda x: 'Male' if x==1 else 'Female')
customers.Marital_status = customers.Marital_status.apply(lambda x: 'Married' if x==1 else 'Alone')
customers

,Cust_ID,Age,Gender,Marital_status,City,Customer_Segment
0,CC25034,35,Male,Alone,Saint Petersburg,Diamond
1,CC59625,22,Male,Married,Moscow,Diamond
2,CC69314,33,Male,Alone,Moscow,Diamond
3,CC67036,46,Male,Married,Saint Petersburg,Diamond
4,CC25597,44,Female,Married,Moscow,Diamond
...,...,...,...,...,...,...
5669,CC53430,27,Male,Married,Moscow,Platinum
5670,CC31823,33,Female,Alone,Moscow,Platinum
5671,CC15335,28,Male,Alone,Moscow,Platinum
5672,CC81556,26,Male,Alone,Moscow,Platinum


In [63]:
#Сохраним обработанные таблицы в локальный источники
transactions.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/transactions.csv', index= False)
customers.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/customers.csv', index= False)
card_base.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/card_base.csv', index= False)
fraud.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/fraud.csv', index= False)
card_tarrif.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/tarrifs.csv', index= False)

In [64]:
#Готовим данные к расчёту доходов. Соединяем массив с транзакциями с данными по лимиту и беспроцентному периоду
transactions = transactions.merge(card_base[['Card_Number','Card_Family','Credit_Limit']], 
                                  left_on='Credit_Card_ID', right_on='Card_Number')
transactions = transactions.merge(card_tarrif[['Card_Family','no % period']], 
                                  left_on = 'Card_Family',right_on = 'Card_Family')

#Меняем знак у операций списания для верного расчёта использования кредитного лимита
transactions['Transaction_Value'][(transactions['Operation_type']=='Payment') 
                                  | (transactions['Operation_type']=='Withdraw')] = transactions['Transaction_Value']*(-1) 
transactions

C:\Users\Asus\AppData\Local\Temp\ipykernel_12612\726715857.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['Transaction_Value'][(transactions['Operation_type']=='Payment')


,Transaction_ID,Oper_date,Credit_Card_ID,Transaction_Value,Operation_type,Category,Card_Number,Card_Family,Credit_Limit,no % period
0,CTID28830551,2016-04-24,1629-9566-3285-2123,-23649,Payment,medicine,1629-9566-3285-2123,Platinum,194000,180
1,CTID52843489,2016-01-16,1629-9566-3285-2123,48307,Refill,,1629-9566-3285-2123,Platinum,194000,180
2,CTID54901377,2016-04-27,1629-9566-3285-2123,-14032,Payment,supermarkets,1629-9566-3285-2123,Platinum,194000,180
3,CTID47600881,2016-07-05,1629-9566-3285-2123,-11063,Payment,transport,1629-9566-3285-2123,Platinum,194000,180
4,CTID15312006,2016-10-30,1629-9566-3285-2123,47956,Refill,,1629-9566-3285-2123,Platinum,194000,180
...,...,...,...,...,...,...,...,...,...,...
9995,CTID58616413,2016-05-17,3991-7326-9292-2844,-36366,Withdraw,,3991-7326-9292-2844,Premium,128000,50
9996,CTID88304220,2016-04-04,3991-7326-9292-2844,-48137,Payment,"house, renovation",3991-7326-9292-2844,Premium,128000,50
9997,CTID61013755,2016-08-07,3991-7326-9292-2844,26303,Refill,,3991-7326-9292-2844,Premium,128000,50
9998,CTID56224624,2016-01-20,3991-7326-9292-2844,-3706,Withdraw,,3991-7326-9292-2844,Premium,128000,50


In [65]:
transactions['final_date'] = ''
transactions['period']=''
transactions['debt']=''
card_lst = list(set(card_base['Card_Number']))
card_lst_short = list(card_base['Card_Number'])[0:1]
final = ''
g = 1
final_data_trans = pd.DataFrame()
for c in card_lst:
    trans_data = transactions[transactions['Credit_Card_ID']==c].sort_values(by='Oper_date',ascending = True)
    final = ''
    for i in range(len(trans_data['Transaction_ID'])):
        if final=='':
            trans_data.iloc[i,10] = trans_data.iloc[i,1]+timedelta(days = int(trans_data.iloc[i,9]))
            trans_data.iloc[i,11] = str(g)
            final = trans_data.iloc[i,1]+timedelta(days = int(trans_data.iloc[i,9]))
        elif (trans_data.iloc[i,1] > final) and (trans_data.iloc[i,3]<0):
            trans_data.iloc[i,10] = trans_data.iloc[i,1]+timedelta(days = int(trans_data.iloc[i,9]))
            final = trans_data.iloc[i,1]+timedelta(days = int(trans_data.iloc[i,9]))
            g = g+1
            trans_data.iloc[i,11] = str(g)
        else:
            trans_data.iloc[i,10] = final
            trans_data.iloc[i,11] = str(g)
    trans_data['debt'] = trans_data['Transaction_Value'].cumsum()
    final_data_trans = pd.concat([final_data_trans,trans_data], axis=0)
    g = 1
final_data_trans = final_data_trans.merge(card_tarrif[['Card_Family','Interest rate']], 
                                          left_on='Card_Family',right_on='Card_Family')
final_data_trans.head(10)

,Transaction_ID,Oper_date,Credit_Card_ID,Transaction_Value,Operation_type,Category,Card_Number,Card_Family,Credit_Limit,no % period,final_date,period,debt,Interest rate
0,CTID35454209,2016-01-25,9248-9032-8076-1225,-10816,Payment,supermarkets,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,-10816,0.17
1,CTID74122546,2016-02-16,9248-9032-8076-1225,49930,Refill,,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,39114,0.17
2,CTID32883588,2016-03-28,9248-9032-8076-1225,1054,Refill,,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,40168,0.17
3,CTID34692836,2016-06-07,9248-9032-8076-1225,49831,Refill,,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,89999,0.17
4,CTID71644803,2016-08-08,9248-9032-8076-1225,35803,Refill,,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,125802,0.17
5,CTID83354271,2016-08-21,9248-9032-8076-1225,31165,Refill,,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,156967,0.17
6,CTID57796271,2016-08-22,9248-9032-8076-1225,49939,Refill,,9248-9032-8076-1225,Gold,25000,120,2016-05-24 00:00:00,1,206906,0.17
7,CTID42853588,2016-11-04,9248-9032-8076-1225,-39295,Payment,"house, renovation",9248-9032-8076-1225,Gold,25000,120,2017-03-04 00:00:00,2,167611,0.17
8,CTID11149009,2016-11-06,9248-9032-8076-1225,-6946,Payment,medicine,9248-9032-8076-1225,Gold,25000,120,2017-03-04 00:00:00,2,160665,0.17
9,CTID69326100,2016-12-11,9248-9032-8076-1225,-44798,Payment,medicine,9248-9032-8076-1225,Gold,25000,120,2017-03-04 00:00:00,2,115867,0.17


In [66]:
#Формируем пустые словари для будущего наполнения данными о доходе
card_lst = list(set(final_data_trans['Credit_Card_ID']))
final_int_income = dict()
final_maint_income = dict()
unique_id = 'CT-'

for c in card_lst:
    #Формируем массивы по нужным нам параметрам, относящимся к одной карте из всего списка
    income = final_data_trans[final_data_trans['Credit_Card_ID']==c]
    income = income[['Credit_Card_ID','Credit_Limit','Oper_date','Operation_type',
                     'Card_Family','Transaction_Value','period','final_date','Interest rate']]
    date = list(income['Oper_date'])
    operations = list(income['Transaction_Value'])
    period = list(income['period'])
    final_periods = list(income['final_date'])
    rate = list(set(income['Interest rate']))[0]
    withdraw_opers = list(income['Transaction_Value'][income['Operation_type']=='Withdraw'])
    withdraw_opers = list(map(lambda x: abs(x),withdraw_opers))
    withdraw_opers_dates = list(income['Oper_date'][income['Operation_type']=='Withdraw'])
    debt = 0
    int_income = 0
    inc = []
    inc_dates = []
    withdraw_income = []
    credit_limit = list(set(income['Credit_Limit']))[0]
    
    #Считаем остаток на счетах по истечению беспроцентного периода и накопленную сумму долга
    #Считаем сумму накопленных процентов, которые пойдут в доход банку
    for n in range(len(income)):
        if n==0:
            debt = operations[n]
        elif n==(len(income)-1):
            if debt<0:
                td = dt.datetime(2017,1,1)-final_periods[n-1]
                td = td.days
                multiple = (1+rate/365)**(td)
                interest_income = debt*multiple - debt
                int_income = interest_income+int_income
                inc.append(int_income)
                inc_dates.append(date[n])
                debt = debt + interest_income + operations[n]
            else:
                debt = debt + operations[n] 
        elif (period[n] != period[n-1]):
            if debt<0:
                td = date[n]-final_periods[n-1]
                td = td.days
                multiple = (1+rate/365)**(td)
                interest_income = debt*multiple - debt
                int_income = interest_income+int_income
                inc.append(int_income)
                inc_dates.append(date[n])
                debt = debt + interest_income + operations[n]
            else:
                debt = debt + operations[n]
        else :
            if debt<0:
                td = date[n]-date[n-1]
                td = td.days
                multiple = (1+rate/365)**(td)
                interest_income = debt*multiple - debt
                int_income = interest_income+int_income
                inc.append(int_income)
                inc_dates.append(date[n])
                debt = debt + interest_income + operations[n]
            else:
                debt = debt+operations[n]
        
    #Считаем доходы по процентам и добавляем в итоговую таблицу доходов
    k = 0
    for i in inc:
        final_int_income[unique_id
                         +str(random.randint(0,1000000))
                         +random.choice(string.ascii_letters)
                         +random.choice(string.ascii_letters)
                         +random.choice(string.ascii_letters)] = [inc_dates[k],c,i,0]
        k = k+1

    #Считаем доходы по снятию наличных и добавляем в итоговую таблицу доходов
    t = income['Card_Family'].iloc[0]
    withdraw_pers = float(card_tarrif[card_tarrif['Card_Family']==t].iloc[0,3])
    withdraw_rub = int(card_tarrif[card_tarrif['Card_Family']==t].iloc[0,4])

    withdraw_income = list(map(lambda x: x*withdraw_pers+withdraw_rub,withdraw_opers))

    for i in range(len(withdraw_income)):
        final_int_income[unique_id
                         +str(random.randint(0,1000000))
                         +random.choice(string.ascii_letters)
                         +random.choice(string.ascii_letters)
                         +random.choice(string.ascii_letters)] = [withdraw_opers_dates[i],c,0,withdraw_income[i]]
    
    #Считаем количество превышений кредитного лимита
    over_limit = income[['period','Transaction_Value']][income['Operation_type']!="Refill"].groupby('period').sum()
    cnt_over_limit = 0
    for i in list(abs(over_limit['Transaction_Value'])):
        if i > credit_limit:
            cnt_over_limit=cnt_over_limit+1
        else:
            pass
    cnt_over_limit = str(cnt_over_limit) + "/" + str(max(list(income['period'])))
    
    #Считаем стоимость обслуживания
    income_maint = income[['Credit_Card_ID','Credit_Limit','Card_Family','Operation_type','Transaction_Value']].merge(card_tarrif[['Card_Family','Maintenance (year)','Withdraw %','Withdraw rub']], left_on = 'Card_Family', right_on = 'Card_Family')
    maint_income = list(set(income_maint['Maintenance (year)']))[0]
    index_ = ['Maintenance income','Over limit (times)']
    final_maint_income[c] = [maint_income,cnt_over_limit]
        


In [67]:
#Длеаем финальные преобразования полученных данных, формируя из словарей датафреймы и приводя в данные в нужный формат
final_int_income = pd.DataFrame(final_int_income).T
final_int_income = final_int_income.reset_index()
final_int_income = final_int_income.rename(columns={'index':'id',0:'date',1:'card',2:'interest_income',3:'withdraw_income'})
final_int_income['interest_income'] = abs(final_int_income['interest_income'].astype(float))
final_int_income['interest_income'] = final_int_income['interest_income'].apply(lambda x: round(x,2))
final_int_income['withdraw_income'] = abs(final_int_income['withdraw_income'].astype(float))
final_int_income['withdraw_income'] = final_int_income['withdraw_income'].apply(lambda x: round(x,2))
final_int_income = final_int_income.sort_values(by = 'date')
final_int_income = final_int_income[['id','date','card','interest_income','withdraw_income']]

final_maint_income = pd.DataFrame(final_maint_income, index = index_)
final_maint_income = final_maint_income.T
final_maint_income = final_maint_income.reset_index()
final_maint_income = final_maint_income.rename(columns = {'index':'card'})

In [68]:
final_int_income.head(10)

,id,date,card,interest_income,withdraw_income
2552,CT-966607Svg,2016-01-01,3134-9713-5764-8285,0.00,1466.03
2339,CT-438516zJD,2016-01-01,5991-4421-8476-3804,0.00,787.35
2024,CT-452575CKr,2016-01-02,5618-9718-9367-2102,11.25,0.00
2956,CT-783093WpN,2016-01-03,6025-2610-4001-6680,0.00,1259.53
4062,CT-318894IRg,2016-01-03,5936-1967-4966-6077,0.00,3030.06
3118,CT-818841PGh,2016-01-03,3573-2839-9869-7747,0.00,3039.62
397,CT-499649cWa,2016-01-03,9540-8558-5897-5046,0.00,645.75
3008,CT-880802zEq,2016-01-03,6045-2636-8278-8875,0.00,2124.00
1418,CT-571654YHz,2016-01-04,6415-7043-2998-3377,0.00,350.87
2529,CT-997057Pyc,2016-01-04,7815-7711-6866-1320,23.82,0.00


In [69]:
final_maint_income.head(10)

,card,Maintenance income,Over limit (times)
0,9248-9032-8076-1225,99,1/2
1,3697-6001-4909-5350,99,3/3
2,2830-1922-3225-5423,0,0/4
3,7276-7349-6334-3809,0,0/4
4,8590-6850-6104-5846,99,2/3
5,5092-6820-3830-6560,0,0/2
6,6531-1690-4324-2760,0,0/4
7,4843-3880-6792-2259,0,0/5
8,9542-6082-9630-9970,199,0/2
9,2524-4184-5908-6750,199,0/2


In [70]:
#Загружаем данные в
final_int_income.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/income.csv',index=False)
final_maint_income.to_csv('C:/Users/Asus/OneDrive/Рабочий стол/Проект для портфолио/Final/maintainance.csv',index=False)